In [13]:
# 使用 python-dotenv 加载 .env 文件
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage

import os

# 加载当前目录下的 .env 文件
load_dotenv(override=True)

# 获取环境变量
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
model_name = os.getenv("MODEL_NAME")

print("API_KEY=", api_key)
print("BASE_URL=", base_url)
print("MODEL_NAME=", model_name)

# 初始化模型
model = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
    temperature=1,
)

# stream = model.stream(
#     [
#         HumanMessage(
#             content="你是什么模型"
#         ),
#     ]
# )

# result = ""

# for chunk in stream:
#     content = chunk.content
#     result += content
#     print(content, end="")  # 不换行

API_KEY= sk-a1d5476da3ea4a169ab5c10e2b424a25
BASE_URL= https://dashscope.aliyuncs.com/compatible-mode/v1
MODEL_NAME= deepseek-v3.2-exp


In [18]:
system_prompt = SystemMessage(
    content="""你是一个智能的 AI 代理，专门用于生成精确的 Python 代码。
请严格遵守以下指令：

如果问题不需要编写代码，给出清晰简洁的答案，不要生成任何代码。

无论被问到什么问题，都只生成基于该问题的 Python 代码，只生成代码，不要在代码前后进行任何解释。

在写代码之前，按步骤（step by step）思考如何解决问题。

如果代码涉及系统级任务，代码应包含识别运行平台和 $HOME 目录的逻辑，以确保代码能成功执行。

代码必须放在 python 代码块内。

确保所有需要的 import 都已包含，以便能执行任务。

在生成代码的末尾，始终包含一行用于运行所生成函数的代码。

代码应以人类可读且易于理解的格式打印输出。

如果生成图表、图形或任何可视化内容，请将其转换为图片并保存到 ./ 位置，同时返回并打印该图片文件路径的文件名。"""
)

user_query = """帮我分析一下./data.csv文件，并生成该数据的描述性统计信息和可视化图表，然后将图表保存为图片文件，最后返回图片文件的路径。"""

stream = model.stream(
    [
        system_prompt,
        HumanMessage(content=user_query),
    ]
)

result = ""

for chunk in stream:
    content = chunk.content
    result += content
    print(content, end="")

# 使用正则模块 提取python代码
import re


def get_python_code(llm_response: str) -> str:
    code_match = re.search(r"```python\n(.*?)```", llm_response, re.DOTALL)

    return code_match.group(1)


def run_python_code_local(code: str) -> str:
    # 使用exec运行代码
    try:
        import subprocess

        result = subprocess.run(
            ["python", "-c", code], capture_output=True, text=True, timeout=10
        )
        return result.stdout if result.returncode == 0 else result.stderr
    except Exception as e:
        print("运行代码时出错:", e)

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

def analyze_data_and_visualize():
    # 检查文件是否存在
    file_path = './data.csv'
    if not os.path.exists(file_path):
        print(f"文件 {file_path} 不存在")
        return None
    
    # 读取数据
    try:
        data = pd.read_csv(file_path)
    except Exception as e:
        print(f"读取文件时出错: {e}")
        return None
    
    # 显示数据基本信息
    print("数据基本信息:")
    print(f"数据形状: {data.shape}")
    print("\n前5行数据:")
    print(data.head())
    print("\n数据类型:")
    print(data.dtypes)
    print("\n缺失值统计:")
    print(data.isnull().sum())
    
    # 描述性统计
    print("\n描述性统计信息:")
    print(data.describe())
    
    # 创建可视化图表
    plt.figure(figsize=(15, 12))
    
    # 数值型数据的直方图
    numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
    
    if len(numeric_columns) > 0:
        # 直方图
        plt.subplot(2, 2, 1)
        for col in numeric_columns[:4]:  # 最多显示4个数值列
            plt.hist(

In [20]:
python_code = get_python_code(result)
code_result = run_python_code_local(python_code)


print("\n\n代码运行结果:\n", code_result)

# 拿到代码的运行结果后，在交给ai做进一步的分析

system_prompt2 = HumanMessage(
    content=f"""
你是一位资深的数据分析师。你刚刚编写并执行了 Python 代码来分析用户的数据。

现在，请根据代码执行的**输出结果**，回答用户的原始问题，并提供深入的业务见解。

### 上下文信息
1. **用户原始问题**: {user_query}
2. **执行的代码**: 
{python_code}
3. **代码执行输出结果**: 
{code_result}

### 分析要求
请根据以上信息，用**中文**撰写一份分析总结，包含以下内容：

1. **数据概览**：简要解读代码输出的统计数据（如平均值、最大最小值、缺失情况等）。不要只是罗列数字，要解释这些数字意味着什么。
2. **图表解读**：如果生成了图表，请结合图表的文件名或类型，解释图表展示了什么趋势、分布或关系。
3. **核心发现**：基于数据和图表，你发现了哪些值得关注的趋势、异常点或有趣的现象？
4. **结论与建议**：直接回答用户的原始问题，并给出基于数据的后续建议。

### 格式要求
- 使用 Markdown 格式。
- 语言通俗易懂，条理清晰。
- 如果有图表，请在合适的位置引用图表（例如："如图 `output.png` 所示..."）。
- **不要**再生成 Python 代码，只提供分析文本。
"""
)

stream = model.stream([system_prompt2])

for chunk in stream:
    content = chunk.content
    result += content
    print(content, end="")



代码运行结果:
 数据基本信息:
数据形状: (200, 5)

前5行数据:
  商品类型     商品名称      销售金额  销售数量      零售价
0   西装   商务修身西装  33125.25    29  1142.25
1   西装  意式双排扣西装  81787.78    38  2152.31
2   衬衫   真丝印花衬衫   8283.60    40   207.09
3  连衣裙  碎花雪纺连衣裙  30221.47    47   643.01
4  连衣裙  碎花雪纺连衣裙  26400.32    32   825.01

数据类型:
商品类型     object
商品名称     object
销售金额    float64
销售数量      int64
零售价     float64
dtype: object

缺失值统计:
商品类型    0
商品名称    0
销售金额    0
销售数量    0
零售价     0
dtype: int64

描述性统计信息:
                销售金额        销售数量          零售价
count     200.000000  200.000000   200.000000
mean    19706.591350   25.975000   791.520200
std     20938.970941   14.909252   649.123653
min       278.460000    1.000000    62.830000
25%      6188.227500   13.000000   359.447500
50%     13055.860000   23.500000   595.510000
75%     25889.872500   40.000000  1080.725000
max    130351.320000   50.000000  2962.530000

可视化图表已保存到: ./data_analysis_plot.png

# 数据分析报告

## 数据概览

根据分析结果，您的数据文件包含了200条销售记录，涵盖5个字段：商品类型、商品名称、销售金额、销售数量和零售价。

*